In [1]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1


In [2]:
!pip install datasets --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 38.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.19.2
    Uninstalling datasets-2.19.2:
      Successfully uninstalled datasets-2.19.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 req

In [3]:
!pip install zipfile

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [4]:
import os
import pandas as pd
import wandb
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from transformers import DataCollatorForSeq2Seq
from datasets import Dataset

2024-06-16 14:32:48.939027: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 14:32:48.939155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 14:32:49.067767: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
epoch_number = 1

In [6]:
os.environ['WANDB_SILENT'] = 'true'

In [7]:
# Device Selection
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
model_checkpoint = 'google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
# Read Data From Excel Files
train_df = pd.read_excel(r'/kaggle/input/new-summary-flow-data/TrainData.xlsx')
val_df = pd.read_excel(r'/kaggle/input/new-summary-flow-data/ValidationData.xlsx')

In [10]:
# Data Preprocessing
def preprocess_text(text):
    return text.strip("[]").replace("'", "").replace("\"", "")

train_df['summary'] = train_df['summary'].apply(preprocess_text)
val_df['summary'] = val_df['summary'].apply(preprocess_text)

In [11]:
# Put The Data In Form Of Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [12]:
# Data Preprocessing
def get_feature(batch):
    encodings = tokenizer(batch['text'], text_target=batch['summary'], max_length=1024, truncation=True)
    encodings = {'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask'], 'labels': encodings['labels']}
    return encodings

train_dataset = train_dataset.map(get_feature, batched=True)
val_dataset = val_dataset.map(get_feature, batched=True)

Map:   0%|          | 0/110374 [00:00<?, ? examples/s]

Map:   0%|          | 0/14411 [00:00<?, ? examples/s]

In [13]:
# Put The Data In Form Of Torch Tensors
columns = ['input_ids', 'labels', 'attention_mask']
train_dataset.set_format(type='torch', columns=columns)
val_dataset.set_format(type='torch', columns=columns)

In [14]:
# Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
training_args = TrainingArguments(
    output_dir='/kaggle/working/T5-model',  # Output directory for checkpoints and evaluation results
    overwrite_output_dir=False,  # Do not overwrite the output directory
    num_train_epochs=1,  # Number of training epochs to run
    per_device_train_batch_size=1,  # Batch size per GPU
    per_device_eval_batch_size=1,  # Batch size for evaluation per GPU
    warmup_steps=500,  # Number of warmup steps
    weight_decay=0.01,  # Weight decay for regularization
    logging_steps=10,  # Log every N steps
    evaluation_strategy='steps',  # Evaluate every `eval_steps` steps
    eval_steps=500,  # Number of steps between evaluations
    save_steps=3000,  # Save checkpoint every N steps
    gradient_accumulation_steps=16,  # Number of gradient accumulation steps
    load_best_model_at_end=True,  # Load the best model from the checkpoint at the end of training
    metric_for_best_model="eval_loss",  # Metric to use for determining the best model
    greater_is_better=False,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# Use GPU
model = model.to(device)

# Trainer Initialization
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [17]:
wandb.login(key="d4d3b2430fb32f105f7640e67c1d04be00ea9c11")

True

In [18]:
# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
500,3.038300,2.852500
1000,2.984800,2.816477
1500,2.868900,2.804408
2000,2.942200,2.794043
2500,2.918600,2.790679
3000,2.850400,2.787938


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=3449, training_loss=2.9623750607564365, metrics={'train_runtime': 29218.0619, 'train_samples_per_second': 3.778, 'train_steps_per_second': 0.118, 'total_flos': 4.583481115012301e+16, 'train_loss': 2.9623750607564365, 'epoch': 0.9999456393715911})

In [19]:
model_name = f'T5_finetuned_SummaryFlow_Epoch_{epoch_number}'

In [20]:
# Save the trained model
model_save_path = f'/kaggle/working/{model_name}'
trainer.save_model(model_save_path)

In [21]:
epoch_file_path = f'{model_save_path}/epoch.txt'

with open(epoch_file_path, 'w') as f:
    f.write(str(epoch_number))

In [22]:
import zipfile
import os
# Define the folder to zip
folder_to_zip = f'/kaggle/working/{model_name}'

# Define the path for the zip file
zip_file_path = f'/kaggle/working/{model_name}.zip'

# Create a Zip file
with zipfile.ZipFile(zip_file_path, 'w') as zip_file:
    for folder_name, subfolders, file_names in os.walk(folder_to_zip):
        for file_name in file_names:
            file_path = os.path.join(folder_name, file_name)
            zip_file.write(file_path, os.path.relpath(file_path, folder_to_zip))


In [23]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Load the service account credentials
creds = Credentials.from_service_account_file('/kaggle/input/googlekey/cnn3-408602-f8776ea41ea4.json')

# Build the drive service
drive_service = build('drive', 'v3', credentials=creds)

# Create a media file upload object
media = MediaFileUpload(zip_file_path, mimetype='application/zip')

# Create a new file on Google Drive
file_metadata = {
    'name': f'{model_name}.zip',
    'mimeType': 'application/zip'
}
file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

print('Uploaded file with ID {}'.format(file.get('id')))

Uploaded file with ID 1GG66zYU4u-uFTRjxIRURkYBnDJdb7obA


In [24]:
from googleapiclient.errors import HttpError

# After uploading the file...
try:
    def callback(request_id, response, exception):
        if exception:
            # Handle error
            print(exception)
        else:
            print("Permission Id: %s" % response.get('id'))

    batch = drive_service.new_batch_http_request(callback=callback)
    anyone_permission = {
        'type': 'anyone',
        'role': 'reader',
    }
    batch.add(drive_service.permissions().create(
        fileId=file.get('id'),
        body=anyone_permission,
        fields='id',
    ))
    batch.execute()
except HttpError as error:
    print(f'An error occurred: {error}')

Permission Id: anyoneWithLink


In [25]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Load the service account credentials
creds = Credentials.from_service_account_file('/kaggle/input/googlekey/cnn3-408602-f8776ea41ea4.json')

# Build the drive service
drive_service = build('drive', 'v3', credentials=creds)

# List the first 100 files in the service account's Google Drive
results = drive_service.files().list(pageSize=100, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(f'{item["name"]} ({item["id"]})')

Files:
T5_finetuned_SummaryFlow_Epoch_1.zip (1GG66zYU4u-uFTRjxIRURkYBnDJdb7obA)
SummaryFlow_Run_8.zip (1mnp_3qv41lbG6VKanGI4Woj1S6FdehuS)
SummaryFlow_Run_7.zip (1hzfaLL2t_t-sVdjY2a3C3YAuErk7Rh84)
SummaryFlow_Run_6.zip (1NNFk_nMfq9MsNL3MqvkEtLpgO9aQmgv2)
SummaryFlow_Run_5.zip (12xLSVrFEHa6L3-TpVuwOV3hfv9cCL5LQ)
SummaryFlow_Run_4.zip (1gvsKk1h9X9H9cVaqi6cRo4WCiu0duznr)
SummaryFlow_Run4.zip (1lU0YqwpIaSKeUZAXzs9RwTYx8gsw9685)
SummaryFlow_Run4.zip (15t3uaXF4ps87d-VtQXK96v5CyihAZtqz)
SummaryFlow_Run3.zip (1FBW9T6WqvQfye3ZNJ0pzj51A7gLBN7cN)
SummaryFlow_Run.zip (1Lt7nNHffIEldUoFlNdWqqi1cc5K7bEmW)
